# Custom Layer Detection in TensorFlow Models
This notebook demonstrates a risk-based decision approach for handling custom layers in TensorFlow models. It includes functions to check for custom layers and to alert if any are found.

In [2]:
# Import necessary libraries
import logging
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer

2024-06-16 14:32:37.644283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 14:32:37.644314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 14:32:37.645242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Configure logging
logging.basicConfig(level=logging.WARNING)

In [4]:
# Define the function to check for custom layers
def check_for_custom_layers(model):
    standard_layers = {tf.keras.layers.Dense, tf.keras.layers.Conv2D, tf.keras.layers.MaxPooling2D, 
                       tf.keras.layers.Flatten, tf.keras.layers.Dropout, tf.keras.layers.Lambda,
                       tf.keras.layers.InputLayer, tf.keras.layers.ReLU, tf.keras.layers.BatchNormalization,
                       tf.keras.layers.GlobalAveragePooling2D, tf.keras.layers.Reshape, tf.keras.layers.Add,
                       tf.keras.layers.Activation, tf.keras.layers.Concatenate, tf.keras.layers.Multiply,
                       # ... add other standard layers as needed
                       }
    for layer in model.layers:
        if type(layer) not in standard_layers:
            logging.warning(f'Model contains custom layer: {type(layer).__name__}')
            return True
    return False

In [5]:
# Custom layer example for demonstration
class CustomLayer(Layer):
    def call(self, inputs):
        return inputs * 2

In [6]:
# Create a simple model for demonstration
def create_sample_model():
    input_layer = tf.keras.layers.Input(shape=(32, 32, 3))
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = CustomLayer()(x)  # Adding a custom layer
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    return model

In [7]:
# Save the sample model
sample_model = create_sample_model()
model_path = '../models/model_with_custom_layer.h5'
sample_model.save(model_path)

/home/yanni/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Load the model from a file
model = load_model(model_path, custom_objects={'CustomLayer': CustomLayer})

In [9]:
# Check the loaded model for custom layers
contains_custom_layers = check_for_custom_layers(model)

# Output the result
if contains_custom_layers:
    print("The model contains custom layers.")
else:
    print("The model does not contain any custom layers.")

The model contains custom layers.


In [10]:
ciffar_model_with_lambda = load_model("../models/simple-cifar10-v1.h5")

In [12]:
# Check the loaded model for custom layers
contains_custom_layers = check_for_custom_layers(model)

# Output the result
if contains_custom_layers:
    print("The model contains custom layers.")
else:
    print("The model does not contain any custom layers.")

The model contains custom layers.
